In [1]:
# file dependencies
from tacc_stats.pickler.job_stats import Job
import cPickle as pickle
import argparse
from os import listdir

In [2]:
# data manipulation dependencies
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
# directory of all jobs on given date
source_dir = '/oasis/projects/nsf/sys200/tcooper/xsede_stats/comet_pickles/2016-08-21/'

# get list of files in directory
pickled_jobs = [ source_dir+job for job in listdir( source_dir ) ]

In [4]:
# access and open 1000 valid jobs
# append to list jobs
jobs = []
count = 0

for job in pickled_jobs:
    if count < 1000:
        try:
            pickle_file = open(job, 'rb')
            jobid = pickle.load(pickle_file)
            jobs.append(jobid)
            pickle_file.close()
            count += 1
        except:
            next
    else:
        break

In [5]:
jobs_dict = {}

for job in jobs:    
    for host_name, host in job.hosts.iteritems():
        typ='all'
        dev='all'
        if 'all' in typ: typ_keys = host.stats.keys()
        else: typ_keys = typ
            
        for type_name in typ_keys:
            type_data = host.stats[type_name]
                    
            if 'all' in dev: dev_keys = type_data.keys()
            else: dev_keys = dev
                
            for dev_name in dev_keys:                    
                dev_data = type_data[dev_name]
                
                jobs_dict[host.job.id] = {
                    "Host" : host.name,
                    "Device Data": np.asmatrix(dev_data).mean(),
                    "Start Time" : pd.to_datetime(round(host.job.start_time), unit='s'),
                    "End Time" : pd.to_datetime(round(host.job.end_time), unit='s'),
                    "Cycles Stats Collected" : len(host.times),
                    "Type of Statistic" : type_name
                    #"Account" : host.job.acct,
                    #"Times" : [pd.to_datetime(round(time), unit='s')
                    #            for time in host.times],
                }

In [6]:
jobs_dict

{'3829166': {'Cycles Stats Collected': 288,
  'Device Data': 7924581286309.6104,
  'End Time': Timestamp('2016-08-21 12:56:17'),
  'Host': 'comet-25-21',
  'Start Time': Timestamp('2016-08-19 12:56:12'),
  'Type of Statistic': 'intel_hsw_r2pci'},
 '3869042': {'Cycles Stats Collected': 2,
  'Device Data': 642438068149.625,
  'End Time': Timestamp('2016-08-22 04:17:03'),
  'Host': 'comet-16-48',
  'Start Time': Timestamp('2016-08-22 03:51:13'),
  'Type of Statistic': 'intel_hsw_r2pci'},
 '3839422': {'Cycles Stats Collected': 6,
  'Device Data': 1409276305520.2812,
  'End Time': Timestamp('2016-08-21 21:42:35'),
  'Host': 'comet-03-71',
  'Start Time': Timestamp('2016-08-19 21:42:15'),
  'Type of Statistic': 'intel_hsw_r2pci'},
 '3866694': {'Cycles Stats Collected': 4,
  'Device Data': 8943071310.458334,
  'End Time': Timestamp('2016-08-22 05:58:04'),
  'Host': 'comet-10-23',
  'Start Time': Timestamp('2016-08-22 03:19:52'),
  'Type of Statistic': 'intel_hsw_r2pci'},
 '3870866': {'Cycles 

In [8]:
# clear jobs with no stats collected
jobs_dict = {job:stats for job,stats in jobs_dict.iteritems()
             if stats['Cycles Stats Collected'] != 0}
jobs_dict

{'3811357': {'Cycles Stats Collected': 36,
  'Device Data': 3171728392967.9473,
  'End Time': Timestamp('2016-08-21 07:09:17'),
  'Host': 'comet-11-15',
  'Start Time': Timestamp('2016-08-21 00:59:35'),
  'Type of Statistic': 'intel_hsw_r2pci'},
 '3827083': {'Cycles Stats Collected': 288,
  'Device Data': 4591386043033.123,
  'End Time': Timestamp('2016-08-21 10:48:21'),
  'Host': 'comet-19-21',
  'Start Time': Timestamp('2016-08-19 10:48:06'),
  'Type of Statistic': 'intel_hsw_r2pci'},
 '3827089': {'Cycles Stats Collected': 288,
  'Device Data': 3523762447974.9043,
  'End Time': Timestamp('2016-08-21 10:48:21'),
  'Host': 'comet-19-40',
  'Start Time': Timestamp('2016-08-19 10:48:06'),
  'Type of Statistic': 'intel_hsw_r2pci'},
 '3828652': {'Cycles Stats Collected': 288,
  'Device Data': 9411614516277.4551,
  'End Time': Timestamp('2016-08-21 08:56:56'),
  'Host': 'comet-19-67',
  'Start Time': Timestamp('2016-08-19 08:56:53'),
  'Type of Statistic': 'intel_hsw_r2pci'},
 '3828674': {'

In [ ]:
# if stats collected for job
# add job stats from all_stats to jobs_dict
for stat,job in all_stats.iteritems():
    for jobName in job.keys():
        if jobName in jobs_dict.keys():
            print jobName

In [ ]:
# merge all data
df = pd.DataFrame( stats_dict )
df = df.reset_index()

In [ ]:
df.head()

In [ ]:
df = df[["index","intel_hsw","ib",
         # "ib_sw","ib_ext",
        "llite","lnet",
         #"mdc","osc",
         "block","cpu","mem",
         "net","nfs","numa",
         "proc","sysv_shm","tmpfs"]]

In [ ]:
# select and rename stats
df = df.rename( columns={"index": "Job Name",
                         "intel_hsw": "Intel Haswell Processor (per core)",
                         "ib": "Infiniband Usage",
                         #"ib_sw": "Infiniband Usage",
                         #"ib_ext": "Infiniband Usage",
                         "llite": "Lustre Filesystem Usage (per mount)",
                         "lnet": "Lustre Network Usage",
                         #"mdc": "Lustre Network Usage",
                         #"osc": "Lustre Filesystem Usage",
                         "block": "Block Device Statistics (per device)",
                         "cpu": "Scheduler Accounting (per CPU)",
                         "mem": "Memory Usage",
                         "net": "Network Device Usage (per device)",
                         "nfs": "NFS System Usage",
                         "numa": "NUMA Statistics (per socket)",
                         "proc": "Process Specific Data",
                         "sysv_shm": "System Shared Memory Segment Usage",
                         "tmpfs": "RAM-Backed Filesystem Usage (per mount)"
                        })

In [ ]:
# reorder columns
df = df.reindex(columns=["Job Name",
   "Block Device Statistics (per device)",
   "Intel Haswell Processor (per core)",
   "Infiniband Usage",
   #"Infiniband Usage",
   #"Infiniband Usage",
   "Lustre Filesystem Usage (per mount)",
   "Lustre Network Usage",
   #"Lustre Network Usage",
   #"Lustre Filesystem Usage",
   "Memory Usage",
   "Network Device Usage (per device)",
   "NFS System Usage",
   "NUMA Statistics (per socket)",
   "Process Specific Data",
   "RAM-Backed Filesystem Usage (per mount)",
   "Scheduler Accounting (per CPU)",
   "System Shared Memory Segment Usage"])

In [ ]:
df.head()

In [ ]:
df.count()

### Computing Averages

In [ ]:
# copy df to alter values
means_df = df

# compute avg of non-nan values
for row in range(0, len(df.index)):
    for col in range(0, len(df.columns)):
        cell = df.iat[row,col]
        if ( type(cell) == np.ndarray ):
            means_df.iat[row,col] = np.mean(cell)

means_df.head()

In [ ]:
# focus on columns with least non-nan values
actual_df = pd.DataFrame(means_df[['Job Name', 'Intel Haswell Processor (per core)', 
                                   'Scheduler Accounting (per CPU)']].dropna())
actual_df = actual_df.reset_index()[['Job Name', 'Intel Haswell Processor (per core)',
                                     'Scheduler Accounting (per CPU)']]

In [ ]:
actual_df.head()

In [ ]:
# Add column for number of times stats collected
# And column for 